# In this notebook we will create a background changer for videos with a condition that these videos have a greenscreen background
We will use Numpy, OpenCv, MatplotLib and MoviePy. For the ones who doesn't know MoviePy is a Python module for video editing.

### Importing the resources

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import moviepy.editor as movie
import os

### Remeber OpenCv shape attribute return the size as (height, width, depth) and MoviePy size attribute return (width, height) and the OpenCv resize function take a tuple argumet of height and width as (width, height)

In [2]:
#import the video
clip = movie.VideoFileClip("videos/van_damme_greenscreen.mp4")

#print the size of th video
print("Video size: ", clip.size)

#import the image background
background_img = cv2.imread("images/house.jpg")

#print the size of the image
print("Image size: ", background_img.shape)

#resize the image to fit the video
background_img_copy = np.copy(background_img)
background_img_resized = cv2.resize(background_img_copy, (1280, 720), interpolation = cv2.INTER_AREA)

#print the new size of the image
print("New image size: ", background_img_resized.shape)

Video size:  [1280, 720]
Image size:  (768, 1366, 3)
New image size:  (720, 1280, 3)


In [3]:
#make a copy of the clip
clip_copy = clip.copy()

In [4]:
#frame_array = [frame for frame in clip_copy.iter_frames(dtype = "unit8")]

# We are going to generate a sequence of images from the video, change the background of each farme, save the new images as jpg in a folder named image_sequence

In [ ]:
for i, frame in enumerate(clip_copy.iter_frames(dtype = "uint8"), 0):
    #make a copy of the current frame
    imageHsv = np.copy(frame)

    #make a copy of background image
    backgroundImgHsv = np.copy(background_img_resized)

    #now let's change the color space from RGB to HSV
    #remeber MoviePy use RGB color space and the video is imported with MoviePy
    imageHsv = cv2.cvtColor(imageHsv, cv2.COLOR_RGB2HSV)
    
    #converting the color space from BGR to HSV
    backgroundImgHsv = cv2.cvtColor(backgroundImgHsv, cv2.COLOR_BGR2HSV)

    #defining the mask boundaries
    lowerMaskH = np.array([58, 0, 0])
    upperMaskH = np.array([90, 255, 255])

    #define the masked area
    maskedArea = cv2.inRange(imageHsv, lowerMaskH, upperMaskH)

    #mask the current frame
    maskedImageHsv = np.copy(imageHsv)
    maskedImageHsv[maskedArea != 0] = [0, 0, 0]

    #mask the background image
    maskedBackgroundImgHsv = np.copy(backgroundImgHsv)
    maskedBackgroundImgHsv[maskedArea == 0] = [0, 0, 0]

    #and now merge the two images together 
    finalImage = maskedImageHsv + maskedBackgroundImgHsv

    #convert the final image from HSV to BGR
    final_frame = cv2.cvtColor(finalImage, cv2.COLOR_HSV2BGR)
    
    #write the curret frame to the hard disk
    #reneber we need the color space to be BGR for OpenCv
    cv2.imwrite(f"image_sequence/frame{i}.jpg", final_frame)